# Simple qusetta QAOA example

We'll show a very basic construction of a $p = 1$ QAOA circuit in *qusetta* form, and then translate it to qiskit, cirq, and quasar.

In [1]:
import sys; sys.path.append("..")
import qusetta as qs

We'll look at a 3 qubit QAOA example for the Hamiltonian $$H = z_0 z_1 + z_1 z_2$$ where each $z_i \in \{1, -1 \}$. We'll first write our circuit as a big string.

In [2]:
# The first element of the circuit is a column of Hadamards
circuit = "H(0); H(1); H(2); "

# next we have exp(-i z_0 z_1 gamma / 2)
circuit += "CX(0, 1); RZ({gamma})(1); CX(0, 1); "

# now the same for exp(-i z_1 z_2 gamma / 2)
circuit += "CX(1, 2); RZ({gamma})(2); CX(1, 2); "

# now we have a row of x rotations by beta
circuit += "RX({beta})(0); RX({beta})(1); RX({beta})(2)"

That's the circuit. We can format the string to substitute in for `beta` and `gamma`, and then we split around `';'` to get it into qusetta's form.

In [3]:
def qusetta_qaoa_circuit(beta, gamma):
    return circuit.format(beta=beta, gamma=gamma).split("; ")

Let's see what it looks like without substituting any values for `beta` or `gamma`.

In [4]:
print(qusetta_qaoa_circuit("beta", "gamma"))

['H(0)', 'H(1)', 'H(2)', 'CX(0, 1)', 'RZ(gamma)(1)', 'CX(0, 1)', 'CX(1, 2)', 'RZ(gamma)(2)', 'CX(1, 2)', 'RX(beta)(0)', 'RX(beta)(1)', 'RX(beta)(2)']


Next let's substitute `beta` = $\pi/2$ and `gamma` = $\pi/4$.

In [5]:
c = qusetta_qaoa_circuit("PI/2", "PI/4")
print(c)

['H(0)', 'H(1)', 'H(2)', 'CX(0, 1)', 'RZ(PI/4)(1)', 'CX(0, 1)', 'CX(1, 2)', 'RZ(PI/4)(2)', 'CX(1, 2)', 'RX(PI/2)(0)', 'RX(PI/2)(1)', 'RX(PI/2)(2)']


Now we convert it to a cirq circuit.

In [6]:
print(qs.Cirq.from_qusetta(c))

0: ───H───@───────────────@───Rx(0.5π)──────────────────────────────
          │               │
1: ───H───X───Rz(0.25π)───X───@──────────────────────@───Rx(0.5π)───
                              │                      │
2: ───H───────────────────────X──────────Rz(0.25π)───X───Rx(0.5π)───


And to quasar.

In [7]:
print(qs.Quasar.from_qusetta(c))

T  : |0|1|2 |3|4 |5 |6|7 |

q0 : -H-@----@-Rx---------
        |    |            
q1 : -H-X-Rz-X-@-----@-Rx-
               |     |    
q2 : -H--------X--Rz-X-Rx-
                          
T  : |0|1|2 |3|4 |5 |6|7 |



And to qiskit.

In [8]:
print(qs.Qiskit.from_qusetta(c))

     ┌───┐                         ┌───┐    ┌──────────┐┌───┐┌──────────┐
q_0: ┤ H ├─────────────────────────┤ X ├────┤ RZ(pi/4) ├┤ X ├┤ RX(pi/2) ├
     ├───┤┌───┐┌──────────┐┌───┐   └─┬─┘    └──────────┘└─┬─┘├──────────┤
q_1: ┤ H ├┤ X ├┤ RZ(pi/4) ├┤ X ├─────■────────────────────■──┤ RX(pi/2) ├
     ├───┤└─┬─┘└──────────┘└─┬─┘┌──────────┐                 └──────────┘
q_2: ┤ H ├──■────────────────■──┤ RX(pi/2) ├─────────────────────────────
     └───┘                      └──────────┘                             


Notice how the bits are reversed in the qiskit circuit. Qiskit orders their qubits differently than everyone else, so qusetta automatically flips them so that the output probability vector of the qiskit circuit equivalent to that of the cirq and quasar circuit. See the README or the `qusetta.Qiskit` docstring for more info.